# Импорт библиотек

нужны для заглушек

In [3]:
import pandas as pd
from dataclasses import dataclass
from pprint import pprint

нужны для получения данных

In [4]:
import requests
import json

# Пример работы с сервисом

Для начала работы с примером нужно локально запустить сервис либо на сервере (тогда надо будет поменять base_url)

In [52]:
base_url = 'http://localhost:5051/forecast/api'

## Заглушки

тут я написал классы для подобия работы с БД, им не нужно уделять внимания

In [5]:
@dataclass
class TimeSeriesDTO:
    """Некий DTO для общения с БД"""
    values: list[float]
    dates: list[str]


class DataLake:
    """Подобие слоя для работы с озером даных"""
    
    @staticmethod
    def get_data_by_id(dataset_uuid: str) -> TimeSeriesDTO:
        """
        подобие функции, которая по id возвращает временной ряд признака из озера данных
        
        просто открывает csv по ключу
        """
        
        tmp = pd.read_csv(f'./raw_data/{dataset_uuid}.csv', sep=';')
        
        return TimeSeriesDTO(
            values=list(tmp['values']),
            dates=list(tmp['date'])
        )

# 1. Для индекса, который хотите предсказать, нужно получить список требуемых признаков

_Признак - временной ряд из озера данных, который нейросеть использует для предсказания индекса. Например, для индекса промышленного производства (ИПП) это будут новостной индекс, Погрузка на сети РЖД, и так далее_
    
### Получить их можно по GET запросу:
data.vavt.ru/forecast/api/v1/{имя_индекса}/features_list
    
- Например, для индекса промышленного производства (ipp) по GET запросу data.vavt.ru/forecast/api/v1/ipp/features_list будет такой результат:


In [68]:
feature_list = requests.get(url=base_url + '/v1/ipp/features_list').json()['features']

pprint(feature_list)

{'bussines_clim': {'dataset_uuid': '14c74eba-c1a7-4aff-a1e3-0aa473ce8062',
                   'description': 'Индикатор бизнес-климата ЦБ '
                                  '(промышленность), пункты, Россия'},
 'cb_monitor': {'dataset_uuid': '7d38c0d5-6bae-4856-b92c-9858223cfa89',
                'description': 'Оценка изменения спроса на продукцию, товары, '
                               'услуги (промышленность), пункты, Россия'},
 'consumer_price': {'dataset_uuid': 'f0873ac7-c29f-481b-8f47-e11c7b1b7c82',
                    'description': 'Индекс цен на электроэнергию в первой '
                                   'ценовой зоне, рублей за МВт/ч, Россия'},
 'curs': {'dataset_uuid': '5e5ac82a-ab76-4567-b095-92f8064acb51',
          'description': 'Официальный курс доллара США на заданную дату, '
                         'устанавливаемый ежедневно'},
 'interest_rate': {'dataset_uuid': '87d93650-33f7-44b2-96df-6d520fa76c12',
                   'description': 'Базовая ставка - краткосроч

__Схема ответа для всех индексов будет одинаковая, будет меняться число признаков, их название и uuid.__

В ответе вам будет полезен database_uuid и название признаков ("news", "rzd", ... )
Описание ("description") может помочь убедиться, что берете и отдаете нужные данные

# 2. Получаем из озера данных данные каждого индекса

Важно отметить, что вся предобработка данных происходит на нашей стороне. Агрегацию по месяцам и фильтрацию по дате проводить не нужно.

Нам нужно сохранить значения и временные отметки индекса. Все индексы начинаются с разного периода времени, для этого нужно их хранить чтобы правильно отрезать нужный период времени, также для агреггации ежедневных данных в помесячные

In [69]:
ipp_data: TimeSeriesDTO = DataLake.get_data_by_id(
    dataset_uuid="c1c92863-1827-405e-b3e4-dea782f57316"
)

приводим к формату в запросе

In [37]:
ipp = {
    "dates": ipp_data.dates,
    "values": ipp_data.values
}

### Пример как должны выглядеть данные

На примере индекса промышленного производства

In [39]:
pd.DataFrame(ipp)\
    .rename(columns={'values': 'Индекс промышленного производства'})\
    .head(5)

,dates,Индекс промышленного производства
0,31.01.2000,110.6
1,29.02.2000,113.3
2,31.03.2000,107.6
3,30.04.2000,117.7
4,31.05.2000,110.8


In [24]:
features = dict()

for feature in feature_list.keys():
    dataset_uuid = feature_list[feature]['dataset_uuid']
    data: TimeSeriesDTO = DataLake.get_data_by_id(dataset_uuid)
    
    features[feature] = {"dates": data.dates, "values": data.values}

### Другой пример как должны выглядеть данные

На примере новостного индекса ЦБ (news)

In [40]:
pd.DataFrame(features['news'])\
    .rename(columns={'values': 'Новостной индекс ЦБ'})\
    .head(5)

,dates,Новостной индекс ЦБ
0,01.02.2004,1.283101
1,01.03.2004,1.316614
2,01.04.2004,1.544784
3,01.05.2004,1.278623
4,01.06.2004,1.160094


Все временные ряды будут иметь такую структуру в запросе. Важно, чтобы значения и отметки времени были одной длины, иначе вернется ошибка валидации

# 3. Формируем запрос

Для этого нужно от пользователя получить список гиперпараметров модели и доверительный интервал.
Например, для модели CatBoost они могут быть следующими:

```json
{
    "depth": 3,
    "learning_rate": 0.1,
    "l2_leaf_reg": 0.005,
    "iterations": 8
}
```

и доверительный интервал:

```json
{
    "confidence_interval": "90%"    
}
```

In [35]:
catboost_params = {
    "depth": 3,
    "learning_rate": 0.1,
    "l2_leaf_reg": 0.005,
    "iterations": 8
}

confidence_interval = "90%"

In [45]:
url = base_url + '/v1/ipp/catboost'

data = {
    "hparams":             catboost_params,     # Гиперпараметры модели
    "confidence_interval": confidence_interval, # Доверительный интервал
    "ipp":                 ipp,                 # Данные ИПП 
    "features":            features             # Данные признаков
}

# 4. Отправляем запрос

In [50]:
with requests.post(url=url, json=data) as response:
    print(f'{response.status_code = }')
    forecast = response.json()

response.status_code = 200


In [67]:
pprint(forecast)

{'month_1': 103.94390256741266,
 'month_2': 103.68966678051972,
 'month_3': 103.33691362309584,
 'scores': [{'mape': 0.018728240386025725, 'r2_score': 0.4906602436192551},
            {'mape': 0.03939419145058797, 'r2_score': 0.4423353444732381},
            {'mape': 0.03835140336589936, 'r2_score': 0.45997708515783686}]}


In [66]:
print('Прогноз: ')
print(f'  - на 1 месяц: {forecast["month_1"]}')
print(f'  - на 2 месяц: {forecast["month_2"]}')
print(f'  - на 3 месяц: {forecast["month_3"]}')
print()
print('Качество моделей:')
print(f'  - Модель на 1 месяц: {forecast["scores"][0]}')
print(f'  - Модель на 2 месяц: {forecast["scores"][1]}')
print(f'  - Модель на 3 месяц: {forecast["scores"][2]}')

Прогноз: 
  - на 1 месяц: 103.94390256741266
  - на 2 месяц: 103.68966678051972
  - на 3 месяц: 103.33691362309584

Качество моделей:
  - Модель на 1 месяц: {'mape': 0.018728240386025725, 'r2_score': 0.4906602436192551}
  - Модель на 2 месяц: {'mape': 0.03939419145058797, 'r2_score': 0.4423353444732381}
  - Модель на 3 месяц: {'mape': 0.03835140336589936, 'r2_score': 0.45997708515783686}
